In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.metrics import confusion_matrix
from itertools import chain, combinations
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier



In [47]:
X = pd.read_csv("data_preprocessed_python/eda_data.csv")
Y = pd.read_csv("data_preprocessed_python/eda_labels.csv")
X.rename(columns={"Unnamed: 0":"video_num"}, inplace=True)
Y.rename(columns={"Unnamed: 0":"video_num"}, inplace=True)
X.columns
Y.columns

Y_label = pd.DataFrame((Y['Arousal'])) #.astype(int)
# Y_label['AVG_Arousal'] = np.where((Y_label['AVG_Arousal'] < 6.0), np.rint(Y_label['AVG_Arousal']), np.ceil(Y_label['AVG_Arousal']))
# Y_label_avg = pd.DataFrame(Y['AVG_Arousal'])
Y_label
X_feature = X.drop(columns=['video_num','eda_filtered','onsets','peaks','variance','mean_eda'])
X_features = X_feature[X_feature.notnull().all(axis=1)] #dropna(inplace=True)
# X_features = np.array(X_features['std_dev']).reshape(-1,1)
Y_labels = Y_label[X_feature.notnull().all(axis=1)]
# Y_labels['AVG_Arousal'] = Y_labels['AVG_Arousal'].astype(int)
# Y_labels_ex = Y_labels[(Y_labels['Arousal'] <= 3.0) | (Y_labels['Arousal'] >= 6.0)]
Y_labels['classes'] = np.where((Y_labels['Arousal'] <= 3.0), -1, (np.where((Y_labels['Arousal'] <= 6.0), 0, 1)))

# X_features = X_features[Y_labels.classes != 0]
# Y_labels = Y_labels[Y_labels.classes != 0]

print(len(Y_labels[Y_labels.classes == -1]))
print(len(Y_labels[Y_labels.classes == 0]))
print(len(Y_labels[Y_labels.classes == 1]))

217
513
511


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [58]:
model = ExtraTreesClassifier(n_estimators=100)
model.fit(X_features, Y_labels['classes'])
print(model.feature_importances_)
print(X_features.columns)
feature_weights = dict(zip(X_features.columns, model.feature_importances_))
sorted(feature_weights.items(), key=lambda x: x[1], reverse=True)

[0.09015413 0.08584321 0.09134426 0.08759504 0.09507718 0.0911929
 0.08965959 0.0956274  0.08729179 0.09721599 0.0889985 ]
Index(['std_dev', 'max_diff', 'skewness', 'median_eda', 'num_peaks',
       'mean_amp', 'max_amp', 'max_eda', 'min_eda', 'rise_time', 'kurt'],
      dtype='object')


[('rise_time', 0.09721599042941491),
 ('max_eda', 0.09562739802458732),
 ('num_peaks', 0.09507718171119568),
 ('skewness', 0.09134425878031086),
 ('mean_amp', 0.09119290476584849),
 ('std_dev', 0.09015413393235006),
 ('max_amp', 0.08965958870997016),
 ('kurt', 0.08899849838441856),
 ('median_eda', 0.0875950443019232),
 ('min_eda', 0.08729178934667572),
 ('max_diff', 0.0858432116133051)]

In [34]:
def powerset(iterable):
    """
    powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)
    """
    xs = list(iterable)
    # note we return an iterator rather than a list
    return chain.from_iterable(combinations(xs,n) for n in range(len(xs)+1))

feature_subsets = list(map(list, powerset(X_features.columns))) 

In [ ]:
def logreg(X_train, X_test, y_train, y_test):
    scalar = StandardScaler()
    scalar.fit(X_train,y_train)
    scaled_X_train = scalar.transform(X_train)
    scaled_X_test = scalar.transform(X_test)
    print(X_train.columns)
    fs_log_reg = LogisticRegression(C=10).fit(scaled_X_train,y_train)

    #training data accuracy 
    #removing rise time causes much less units to be classified is low arousal
#     print("Training Data: \n")
    fs_y_pred = fs_log_reg.predict(scaled_X_train)
    R_train = recall_score(y_train,fs_y_pred,average=None)
    P_train = precision_score(y_train,fs_y_pred,average=None)
    F1_train = f1_score(y_train,fs_y_pred,average=None)
#     print("Recall: "+str(R_train),"Precision: "+str(P_train), "F1 score: "+str(F1_train))
    score_train = fs_log_reg.score(scaled_X_train,y_train)
#     print(fs_log_reg.coef_, fs_log_reg.score(scaled_X_train,y_train))
    # print(X_features.columns)

    matrix_train = confusion_matrix(y_train, fs_y_pred)
    print(matrix_train)
#     print(matrix)

    #testing data accuracy 
#     print("\n Testing Data: \n")
    fs_y_pred = fs_log_reg.predict(scaled_X_test)
    R_test = recall_score(y_test,fs_y_pred,average=None)
    P_test = precision_score(y_test,fs_y_pred,average=None)
    F1_test = f1_score(y_test,fs_y_pred,average=None)
#     print("Recall: "+str(R_test),"Precision: "+str(P_test), "F1 score: "+str(F1_test))
    score_test = fs_log_reg.score(scaled_X_test,y_test)
#     print(fs_log_reg.coef_, fs_log_reg.score(scaled_X_test,y_test))
    # print(X_features.columns)

    matrix_test = confusion_matrix(y_test, fs_y_pred)
    print(matrix_test)
    
    return score_train, F1_train, score_test, F1_test


In [ ]:
best_acc_features_test = []
best_acc_test = 0.0
best_acc_F1_test = []

best_acc_features_train = []
best_acc_train = 0.0
best_acc_F1_train = []

for sub in feature_subsets:
#     print(sub)
    if not sub or len(sub) < 2:
        continue
    X_features_sub = X_features[sub]
#     print(X_features_sub)
    X_train, X_test, y_train, y_test = train_test_split(X_features_sub, Y_labels['classes'], test_size=0.20, random_state=42)
    score_train, F1_train, score_test, F1_test = logreg(X_train, X_test, y_train, y_test)
    if score_train > best_acc_train:
        best_acc_train = score_train
        best_acc_features_train = sub
        best_acc_F1_train = F1_train
    
    if score_test > best_acc_test:
        best_acc_test = score_test
        best_acc_features_test = sub
        best_acc_F1_test = F1_test

X_features_sub = X_features[best_acc_features_test]
X_train, X_test, y_train, y_test = train_test_split(X_features_sub, Y_labels['classes'], test_size=0.20, random_state=42)
print(logreg(X_train, X_test, y_train, y_test))
    
X_features_sub = X_features[best_acc_features_train]
X_train, X_test, y_train, y_test = train_test_split(X_features_sub, Y_labels['classes'], test_size=0.20, random_state=42)
print(logreg(X_train, X_test, y_train, y_test))